In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
from llama import Workflow, Llama

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29502"

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=8192,
    max_batch_size=8,
    model_parallel_size=1,
    max_nodes=100,
)

llama = Llama(workflow.model, workflow.tokenizer)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 34.17 seconds


In [3]:
import json
with open('/home/tbai4/llama3/data/triviaqa/unfiltered-web-train.json') as f:
    data = json.load(f)

In [6]:
problems = data['Data']

for i, problem in enumerate(problems):
    if i == 10:
        break
    print('###')
    print(problem['Question'])
    print(problem['Answer']['Value'])

###
Who was President when the first Peanuts cartoon was published?
Harry Truman
###
Which American-born Sinclair won the Nobel Prize for Literature in 1930?
Sinclair Lewis
###
Where in England was Dame Judi Dench born?
York
###
William Christensen of Madison, New Jersey, has claimed to have the world's biggest collection of what?
Beer Cans
###
In which decade did Billboard magazine first publish and American hit chart?
30s
###
Where was horse racing's Breeders' Cup held in 1988?
Churchill Downs, Louisville, Kentucky
###
From which country did Angola achieve independence in 1975?
Portugal
###
Which city does David Soul come from?
Chicago
###
Who won Super Bowl XX?
Chicago Bears
###
Which was the first European country to abolish capital punishment?
Norway


In [31]:
from operator import itemgetter as get

def ask_sequential(workflow, subset):
    workflow.reset()
    
    [prompt] = workflow.insert([
        {
            'messages': (
                [{'role': 'system', 'content': 'Answer ALL of the user\'s questions. Answer with an numbered list. Do not include extraneous text.'}] +
                [{'role': 'user', 'content': f'Question {i+1}: {p['Question']}'} for i, p in enumerate(subset)]
            ),
            'parent_ids': []
        }
    ])
    
    [response] = get('tokens')(workflow.step(
        tasks=[{
            'header': ('assistant', None),
            'prefill': '',
            'parent_ids': [prompt['id']],
        }]
    ))
    
    return workflow.tokenizer.decode(response)

def ask_parallel(workflow, subset, annotate=False, compact=False):
    workflow.reset()
    
    [prompt] = workflow.insert([
        {
            'messages': [{'role': 'system', 'content': 'Answer ALL of the user\'s questions. Answer with an numbered list. Do not include extraneous text.'}],
            'parent_ids': []
        }
    ])

    questions = workflow.insert([
        {
            'messages': [{'role': 'user', 'content': f'{f'Question {i+1}: ' if annotate else ''}{p['Question']}'}],
            'parent_ids': [prompt['id']],
        }
        for i, p in enumerate(subset)
    ])
    
    [response] = get('tokens')(workflow.step(
        tasks=[{
            'header': ('assistant', None),
            'prefill': '',
            'parent_ids': [prompt['id']] + [q['id'] for q in questions],
        }],
        compact=compact
    ))
    
    return workflow.tokenizer.decode(response)

def parse_items(text):
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    items = [line.split('. ', 1)[1] if '. ' in line else line for line in lines]
    return items

## N=2, sequential (baseline)

In [37]:
import random
from tqdm import tqdm
from pathlib import Path

output = []
for seed in tqdm(range(30)):
    random.seed(seed)
    output.append(f"\n## Trial {seed}\n")
    subset = random.sample(problems, k=2)
    answer = ask_sequential(workflow, subset)

    for i, (s, answer) in enumerate(zip(subset, parse_items(answer))):
        output.extend([
            f"### Question {i+1}\n",
            f"**Question:** {s['Question']}\n", 
            f"**Ground Truth:** {s['Answer']['Value']}\n",
            f"**Generated:** {answer}\n"
        ])
        if i < len(subset) - 1:
            output.append("\n---\n")

Path('/home/tbai4/llama3/dumps/sequential_n2.md').write_text('\n'.join(output))

100%|██████████| 30/30 [00:23<00:00,  1.28it/s]


11292

## N=2, Parallel

In [38]:
import random
from tqdm import tqdm
from pathlib import Path

configs = [
   ('parallel_base_n2.md', False, False),
   ('parallel_annotated_n2.md', True, False), 
   ('parallel_linearized_n2.md', False, True),
   ('parallel_annotated_linearized_n2.md', True, True)
]

for filename, annotate, compact in configs:
    output = []
    for seed in tqdm(range(30), desc=f"annotate={annotate}, compact={compact}"):
        random.seed(seed)
        output.append(f"\n## Trial {seed}\n")
        subset = random.sample(problems, k=2)
        answer = ask_parallel(workflow, subset, annotate=annotate, compact=compact)

        for i, (s, answer) in enumerate(zip(subset, parse_items(answer))):
            output.extend([
                f"### Question {i+1}\n",
                f"**Question:** {s['Question']}\n", 
                f"**Ground Truth:** {s['Answer']['Value']}\n",
                f"**Generated:** {answer}\n"
            ])
            
        if i < len(subset) - 1:
            output.append("\n---\n")

    Path(f'/home/tbai4/llama3/dumps/triviaqa/{filename}').write_text('\n'.join(output))

annotate=True, compact=True: 100%|██████████| 30/30 [00:18<00:00,  1.64it/s]


## exploratory

In [12]:
res = llama.chat_completion(
    dialogs=[[
        {'role': 'system', 'content': 'Answer ALL of the user\'s questions. Answer with an numbered list. Do not include extraneous text.'},
        {'role': 'user', 'content': 'Which city does David Soul come from?'},
        {'role': 'user', 'content': 'Who was President when the first Peanuts cartoon was published?'},
        {'role': 'user', 'content': 'From which country did Angola achieve independence in 1975?'},
        {'role': 'user', 'content': 'Who won Super Bowl XX?'},
    ]]
)

print(res[0]['generation']['content'])

1. Chicago
2. Richard Nixon
3. Portugal
4. Chicago Bears


In [66]:
from operator import itemgetter as get

messages = [
    {'role': 'system', 'content': 'Answer ALL of the user\'s questions. Answer with an numbered list. Do not include extraneous text.'},
    {'role': 'user', 'content': 'Which city does David Soul come from?'},
    {'role': 'user', 'content': 'Who was President when the first Peanuts cartoon was published?'},
    {'role': 'user', 'content': 'From which country did Angola achieve independence in 1975?'},
    {'role': 'user', 'content': 'Who won Super Bowl XX?'}
]

[prompt] = workflow.insert([{'messages': messages, 'parent_ids': []}])

[response] = get('tokens')(workflow.step(
    [{
        'header': ('assistant', None),
        'prefill': '',
        'parent_ids': [prompt['id']]
    }]
))

print(workflow.tokenizer.decode(response))

1. Chicago
2. Richard Nixon
3. Portugal
4. Chicago Bears


In [76]:
from operator import itemgetter as get

workflow.reset()

[prompt] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer ALL of the user\'s questions. Answer with an numbered list. Do not include extraneous text.'}],
        'parent_ids': []
    }
])

questions = workflow.insert([
        {
            'messages': [{'role': 'user', 'content': 'Question 1: Which city does David Soul come from?'}],
            'parent_ids': [prompt['id']],
        },
        {
            'messages': [{'role': 'user', 'content': 'Question 4: From which country did Angola achieve independence in 1975?'},],
            'parent_ids': [prompt['id']],
        },
        {
            'messages': [{'role': 'user', 'content': 'Question 3: Who won Super Bowl XX?'},],
            'parent_ids': [prompt['id']],
        },
        {
            'messages': [{'role': 'user', 'content': 'Question 2: Who was President when the first Peanuts cartoon was published?'},],
            'parent_ids': [prompt['id']],
        },
])

[response] = get('tokens')(workflow.step(
    tasks=[{
        'header': ('assistant', None),
        'prefill': '',
        'parent_ids': [prompt['id']] + [q['id'] for q in questions],
    }],
    compact=True
))

print(workflow.tokenizer.decode(response))

1. Gerald Ford


In [83]:
from operator import itemgetter as get

workflow.reset()

[prompt] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer ALL of the user\'s questions. Answer with an numbered list. Do not include extraneous text.'}],
        'parent_ids': []
    }
])

questions = workflow.insert([
        {
            'messages': [
                {'role': 'user', 'content': 'Which city does David Soul come from?'},
                {'role': 'user', 'content': 'From which country did Angola achieve independence in 1975?'},
            ],
            'parent_ids': [prompt['id']],
        },
        {
            'messages': [
                {'role': 'user', 'content': 'Who won Super Bowl XX?'},
                {'role': 'user', 'content': 'Who was President when the first Peanuts cartoon was published?'},
            ],
            'parent_ids': [prompt['id']],
        },
])

[response] = get('tokens')(workflow.step(
    tasks=[{
        'header': ('assistant', None),
        'prefill': '',
        'parent_ids': [prompt['id']] + [q['id'] for q in questions],
    }],
))

print(workflow.tokenizer.decode(response))

1. Chicago
2. United States
